In [1]:
import pandas as pd

# =========================
# 1️⃣ 데이터 로드
# =========================
csv_path = "new_flight_data.csv"
df = pd.read_csv(csv_path)

print("✅ CSV 로드 완료")
print("전체 행 수:", len(df))

✅ CSV 로드 완료
전체 행 수: 5744619


In [2]:
# =========================
# 2️⃣ 상태 컬럼 유니크 + 개수 확인
# =========================

# 컬럼 존재 여부 체크
if "상태" not in df.columns:
    print("❌ '상태' 컬럼이 없습니다. 현재 컬럼 목록:")
    print(df.columns.tolist())
else:
    print("\n✅ 상태 컬럼 유니크 값 개수:", df["상태"].nunique(dropna=False))

    # 값별 개수 (NaN 포함)
    status_counts = df["상태"].value_counts(dropna=False)
    print("\n📊 상태 값별 개수 (NaN 포함)")
    display(status_counts)

    # 비율까지 같이 보고 싶으면
    status_ratio = (status_counts / len(df)).rename("ratio")
    status_summary = pd.concat([status_counts.rename("count"), status_ratio], axis=1)
    print("\n📊 상태 값별 개수 + 비율")
    display(status_summary)



✅ 상태 컬럼 유니크 값 개수: 5

📊 상태 값별 개수 (NaN 포함)


정상운항    4844631
지연       841406
취소        37690
NaN       19806
회항         1086
Name: 상태, dtype: int64


📊 상태 값별 개수 + 비율


,count,ratio
정상운항,4844631,0.843334
지연,841406,0.146469
취소,37690,0.006561
NaN,19806,0.003448
회항,1086,0.000189


In [3]:

# 일자 컬럼을 datetime으로 변환
df["일자_dt"] = pd.to_datetime(
    df["일자"].astype(str),
    format="%Y%m%d",
    errors="coerce"
)

print("📅 flight 데이터 날짜 범위")
print("시작일:", df["일자_dt"].min())
print("종료일:", df["일자_dt"].max())

print("\n전체 일수:",
      (df["일자_dt"].max() - df["일자_dt"].min()).days + 1)

📅 flight 데이터 날짜 범위
시작일: 2018-01-01 00:00:00
종료일: 2025-12-31 00:00:00

전체 일수: 2922


In [4]:
# =========================
# 2️⃣ 항공사별 비행 횟수 집계
# =========================
airline_counts = (
    df["항공사"]
    .value_counts(dropna=False)
    .reset_index()
)
airline_counts.columns = ["항공사", "total_flights"]

print("\n✈️ 항공사별 비행 횟수")
print(airline_counts.sort_values("total_flights"))


✈️ 항공사별 비행 횟수
           항공사  total_flights
175  카고룩스 이탈리아              1
170    카타르 아미리              1
171   아시아퍼시픽항공              1
174     비스타젯항공              1
173     천진에어카고              1
..         ...            ...
4        티웨이항공         558157
3          진에어         612568
2         제주항공         742622
1       아시아나항공         816757
0         대한항공        1193584

[176 rows x 2 columns]


In [5]:
# =========================
# 3️⃣ 항공사 24회 이상만 선택
# =========================
airline_counts_24 = airline_counts[
    airline_counts["total_flights"] >= 24
]

valid_airlines = airline_counts_24["항공사"].tolist()

print("\n✅ 사용할 항공사 수:", len(valid_airlines))
print(valid_airlines)


✅ 사용할 항공사 수: 140
['대한항공', '아시아나항공', '제주항공', '진에어', '티웨이항공', '에어부산', '이스타항공', '에어서울', '중국동방항공', '비엣젯항공', '중국남방항공', '중국국제항공', '하이에어', '에어로케이항공', '베트남항공', '산동항공', '춘추항공', '피치항공', '델타항공', '중화항공', '필리핀항공', '싱가폴항공', '캐세이퍼시픽항공', '타이항공', '홍콩익스프레스', '장영항공(長榮)', '일본항공', nan, '필리핀에어아시아', '세부퍼시픽항공', '전일본항공', '에어캐나다', '상해항공', '에어프레미아', '타이거 에어 타이완(스마트캣)', '에어아시아 엑스', '유나이티드항공', '몽골항공', '독일항공', '코리아익스프레스', '심천항공', '타이에어아시아엑스', '중국길상항공', '네덜란드항공', '터키항공', '오로라항공', '에티하드', '에티오피안항공', '에어마카오', '가루다인도네시아', '카타르항공', '프랑스항공', '에미레이트항공', '아메리칸항공', '스쿠트타이거', '말레이시아항공', '샤먼항공', '홍콩항공', '팬퍼시픽항공', '폴란드항공', '에어포항', '핀란드항공', '에어아스타나항공', '라오항공', '우즈베키스탄항공', '청도항공', '에어필립', '천진에어라인', '시베리아 항공', '하와이안항공', '집에어도쿄', '스카이앙코르항공', '사천항공', '아에로멕시코', '에어인디아', '룽에어', '드래곤에어', '젯스타 에어웨이즈', '뱀부항공', '러시아항공', '미얀마국제항공', '말린도항공', '브루나이항공', '영국항공', '대만구항공', '파라타항공', '에어재팬', '에어아시아버하드', '아틀라스화물항공', '젯스타퍼시픽항공', '에어로 몽골리아', '북경수도항공', '콴타스항공', '알이탈리아항공', '스리랑칸항공', '에어뉴질랜드', '체코항공', '사우디아라비아 항공', '야쿠티아항공', '팔익스프레스', '유니항공(立榮)', '오케이

In [6]:
# =========================
# 4️⃣ 항공사 기준 필터링
# =========================
df_filtered = df[df["항공사"].isin(valid_airlines)].reset_index(drop=True)

print("항공사 필터링 후 행 수:", len(df_filtered))

항공사 필터링 후 행 수: 5744412


In [7]:
# =========================
# 5️⃣ 출발지 15개만 사용
# =========================
valid_departures = [
    "인천", "김포", "청주", "양양", "군산",
    "원주", "김해", "제주", "대구", "광주",
    "여수", "울산", "포항경주", "사천", "무안"
]

df_filtered = df_filtered[
    df_filtered["출발지"].isin(valid_departures)
].reset_index(drop=True)

print("출발지 15개 필터링 후 행 수:", len(df_filtered))

출발지 15개 필터링 후 행 수: 4468210


In [8]:
# =========================
# 6️⃣ 출발지 검증
# =========================
print("\n출발지 유니크 개수:", df_filtered["출발지"].nunique())
print(sorted(df_filtered["출발지"].unique()))


출발지 유니크 개수: 15
['광주', '군산', '김포', '김해', '대구', '무안', '사천', '양양', '여수', '울산', '원주', '인천', '제주', '청주', '포항경주']


In [9]:
# =========================
# ⏱ 출발시간 결측 제거 (국내 출발 기준)
# =========================
print("출발시간 결측 제거 전:", len(df_filtered))

df_filtered = df_filtered[df_filtered["출발시간"].notna()].reset_index(drop=True)

print("출발시간 결측 제거 후:", len(df_filtered))


출발시간 결측 제거 전: 4468210
출발시간 결측 제거 후: 2843943


In [10]:
# =========================
# 2️⃣ 상태 컬럼 유니크 + 개수 확인
# =========================

# 컬럼 존재 여부 체크
if "상태" not in df.columns:
    print("❌ '상태' 컬럼이 없습니다. 현재 컬럼 목록:")
    print(df_filtered.columns.tolist())
else:
    print("\n✅ 상태 컬럼 유니크 값 개수:", df["상태"].nunique(dropna=False))

    # 값별 개수 (NaN 포함)
    status_counts = df_filtered["상태"].value_counts(dropna=False)
    print("\n📊 상태 값별 개수 (NaN 포함)")
    display(status_counts)

    # 비율까지 같이 보고 싶으면
    status_ratio = (status_counts / len(df)).rename("ratio")
    status_summary = pd.concat([status_counts.rename("count"), status_ratio], axis=1)
    print("\n📊 상태 값별 개수 + 비율")
    display(status_summary)



✅ 상태 컬럼 유니크 값 개수: 5

📊 상태 값별 개수 (NaN 포함)


정상운항    2360833
지연       481449
회항          859
취소          579
NaN         223
Name: 상태, dtype: int64


📊 상태 값별 개수 + 비율


,count,ratio
정상운항,2360833,0.410964
지연,481449,0.083809
회항,859,0.000150
취소,579,0.000101
NaN,223,0.000039


In [11]:
# =========================
# 🔍 컬럼별 결측치 현황 (EDA)
# =========================

missing_summary = (
    df_filtered
    .isna()
    .sum()
    .reset_index()
)

missing_summary.columns = ["column", "missing_cnt"]
missing_summary["missing_ratio"] = missing_summary["missing_cnt"] / len(df_filtered)
missing_summary["dtype"] = missing_summary["column"].map(
    lambda c: df_filtered[c].dtype
)

# 결측치 있는 컬럼만 보기
missing_summary = missing_summary[missing_summary["missing_cnt"] > 0]

# 결측치 많은 순서로 정렬
missing_summary = missing_summary.sort_values(
    "missing_ratio",
    ascending=False
)

print("📊 컬럼별 결측치 현황")
display(missing_summary)


📊 컬럼별 결측치 현황


,column,missing_cnt,missing_ratio,dtype
14,도착시간,2843943,1.000000e+00,float64
12,지연원인,2391614,8.409500e-01,object
8,예상시간,22395,7.874630e-03,float64
3,항공사,6681,2.349203e-03,object
5,도착지,347,1.220137e-04,object
11,상태,223,7.841226e-05,object
7,계획시간,1,3.516245e-07,float64


In [12]:
# =========================
# 출발 datetime 생성
# =========================

df_filtered["출발시간_clean"] = (
    df_filtered["출발시간"]
    .astype(int)
    .astype(str)
    .str.zfill(4)
)

df_filtered["departure_datetime"] = pd.to_datetime(
    df_filtered["일자"].astype(str) + df_filtered["출발시간_clean"],
    format="%Y%m%d%H%M",
    errors="coerce"
)

print("departure_datetime NaT 개수:", df_filtered["departure_datetime"].isna().sum())


departure_datetime NaT 개수: 0


In [13]:
# =========================
# 시간 파생 변수
# =========================
df_filtered["dep_hour"] = df_filtered["departure_datetime"].dt.hour
df_filtered["dep_weekday"] = df_filtered["departure_datetime"].dt.weekday
df_filtered["is_weekend"] = df_filtered["dep_weekday"].isin([5, 6]).astype(int)


In [14]:
# =========================
# 7️⃣ 타겟 변수 생성 (is_delay)
# =========================
def make_delay_label(x):
    if pd.isna(x):
        return 0
    if any(k in x for k in ["지연", "취소", "회항"]):
        return 1
    return 0

df_filtered["is_delay"] = df_filtered["상태"].apply(make_delay_label)

print("\n⏱ 지연 분포")
print(df_filtered["is_delay"].value_counts())


⏱ 지연 분포
0    2361056
1     482887
Name: is_delay, dtype: int64


In [15]:
# =========================
# 🔟 국내 / 국제 구분 (도착지 기준)
# =========================

domestic_airports = set([
    "인천", "김포", "청주", "양양", "군산",
    "원주", "김해", "제주", "대구", "광주",
    "여수", "울산", "포항경주", "사천", "무안"
])

df_filtered["flight_type"] = df_filtered["도착지"].apply(
    lambda x: "국내" if pd.notna(x) and x.strip() in domestic_airports else "국제"
)

print("\n✈️ 국내 / 국제 분포 (도착지 기준)")
print(df_filtered["flight_type"].value_counts())



✈️ 국내 / 국제 분포 (도착지 기준)
국내    1577185
국제    1266758
Name: flight_type, dtype: int64


In [16]:
# =========================
# 8️⃣ 샘플 1000개 (EDA / 테스트용)
# =========================
df_sample = df_filtered.sample(
    n=min(1000, len(df_filtered)),
    random_state=42
)

df_sample.to_excel(
    "new_flight_data_sample_1000.xlsx",
    index=False
)

print("✅ 샘플 엑셀 저장 완료 (국내/국제 포함)")

PermissionError: [Errno 13] Permission denied: 'new_flight_data_sample_1000.xlsx'

In [ ]:
# =========================
# 9️⃣ 최종 분석용 CSV 저장
# =========================
df_filtered.to_csv(
    "new_flight_analysis_summary.csv",
    index=False,
    encoding="utf-8-sig"
)

print("✅ 최종 분석용 CSV 저장 완료 (국내/국제 포함)")